# Data Processing

The purpose of this notebook is to process the data and keep it separate from the main analysis and visualization. We will write the code unnecessary for the user to interact with such as functions, import libraries, and processing the data into sheets of data we can interpret.

## Table of Contents
* content

## Notebooks
* [Overview Notebook](airpollution.ipynb)
* [Processing Notebook](dataprocessing.ipynb)
* [Analysis Notebook](data-analysis.ipynb)

## Import Libraries

Here are the libraries we are going to use to graph and analyze our data.

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from glob import glob
from matplotlib.pyplot import figure 
import datetime
from datetime import datetime
from datetime import timedelta

## Functions

Here are the functions we have created to analyze the datasets we are looking at and graph them to display trends and comparisons.

In [2]:
def concatinate(y):
    
    path = y
    data = glob(path + '*csv')
    data.sort()
    return pd.concat([pd.read_csv(f) for f in data])

def createDataframe(x):
    
    x['ozone_rolling_mean_365'] = x['Ozone_AQI_Value'].rolling(window=365).mean()
    x['particle_rolling_mean_365'] = x['PM2.5_AQI_Value'].rolling(window=365).mean()
    data = pd.DataFrame({'X' : x['ozone_rolling_mean_365'],
                         'Y' : x['particle_rolling_mean_365'],
                         'T' : x['Date']})
    return data

def time_histogram(x):
    
    pm = x[['Date', 'PM2.5_AQI_Value']]
    pm = pm.dropna()
    pm['Delta_Time'] = pm['Date']
    pm['Delta_Time'] = pd.to_datetime(pm['Delta_Time'], format = '%m/%d/%y')
    #pm['PM2.5_AQI_Value'].diff().plot.hist()
    pm['Delta_Time'] = pm['Delta_Time'].diff()
    pm['Delta_Time'].plot.hist()
        

def aqi_plot(x): 
    
    ax = x.plot.line('T',['X', 'Y'], figsize = (15, 10), title = 'Air Quality Trend')
    ax.set_xlabel('Time (1985-2020)')
    ax.set_ylabel('Air Quality Index (AQI)')
    ax.legend(['Ozone (O3)', 'Particulate Matter (PM2.5)'])
    ax.grid(True)
    
def plot_all(x):
    

    data = x[0]
    
    for f in range (1, len(x)):
        data = pd.merge(data, x[f], how = 'inner', on = 'T')
    ax = data.plot.line('T', ['X_x', 'X_y', 'X'], figsize = (40, 10), title = 'Ozone Quality Trend', colormap = 'Paired')
    ax.set_xlabel('Time (1985-2020)')
    ax.set_ylabel('Average AQI')
    ax.legend(['NY-NJ-PA', 'TX', 'MA-NH', 'CA', 'PA-NJ-DE-MD', 'WA', 'MN_WI', 'IL-IN-WI', 'GA', 
              'MI', 'FL', 'AZ', 'DC-VA-MD-WV', 'CO', 'MO-IL'])
    ax.grid(True)
    
    bx = data.plot.line('T', ['X_y', 'Y_y', 'Y'], figsize = (40, 10), title = 'Particulate Matter Quality Trend', colormap = 'Paired')
    bx.set_xlabel('Time (1985-2020)')
    bx.set_ylabel('Average AQI')
    bx.legend(['NY-NJ-PA', 'TX', 'MA-NH', 'CA', 'PA-NJ-DE-MD', 'WA', 'MN-WI', 'IL-IN-WI', 'GA', 
              'MI', 'FL', 'AZ', 'DC-VA-MD-WV', 'CO', 'MO-IL'])
    bx.grid(True)

def production(x, z):

    y = x
    goods = ['Gasoline', 'Fuel oils', 'Coal n.e.c', 'Motorized vehicles', 'Transport equipment', 'Machinery'
                   , 'Chemical products', 'Newsprint/paper', 'Printed products']

    y = y.sort_values(by = ['ID SCTG2', 'Year'], ascending = True)

    gasoline_index = [y.loc[y['SCTG2'] == 'Gasoline'].index.values.tolist()]
    gasoline = []
    for i in range (5):
        gasoline.append(x.iat[gasoline_index[0][i], 4])
    
    fuel_oils_index = [y.loc[y['SCTG2'] == 'Fuel oils'].index.values.tolist()]
    fuel_oils = []
    for i in range (5):
        fuel_oils.append(x.iat[fuel_oils_index[0][i], 4])
        
    coal_nec_index = [y.loc[y['SCTG2'] == 'Coal-n.e.c.'].index.values.tolist()]
    coal_nec = []
    for i in range (5):
        coal_nec.append(x.iat[coal_nec_index[0][i], 4])

    motorized_vehicles_index = [y.loc[y['SCTG2'] == 'Motorized vehicles'].index.values.tolist()]
    motorized_vehicles = []
    for i in range (5):
        motorized_vehicles.append(x.iat[motorized_vehicles_index[0][i], 4])
    
    transport_equipment_index = [y.loc[y['SCTG2'] == 'Transport equip.'].index.values.tolist()]
    transport_equipment = []
    for i in range (5):
        transport_equipment.append(x.iat[transport_equipment_index[0][i], 4])
    
    machinery_index = [y.loc[y['SCTG2'] == 'Machinery'].index.values.tolist()]
    machinery = []
    for i in range (5):
        machinery.append(x.iat[machinery_index[0][i], 4])
        
    
    newsprint_paper_index = [y.loc[y['SCTG2'] == 'Newsprint/paper'].index.values.tolist()]
    newsprint_paper = []
    for i in range (5):
        newsprint_paper.append(x.iat[newsprint_paper_index[0][i], 4])
    
    printed_products_index = [y.loc[y['SCTG2'] == 'Printed prods.'].index.values.tolist()]
    printed_products = []
    for i in range (5):
        printed_products.append(x.iat[printed_products_index[0][i], 4])
        
    time = ['2012', '2013', '2014', '2015', '2020']
    
    df = pd.DataFrame({'Year' : time,
                       'Gasoline' : gasoline, 
                       'Fuel oils' : fuel_oils,
                       'Coal n.e.c.' : coal_nec,
                       'Motorized vehicles' : motorized_vehicles,
                       'Transport equipment' : transport_equipment,
                       'Machinery' : machinery,
                       'Newsprint paper' : newsprint_paper,
                       'Printed products' : printed_products})
    
    #ax = df.plot.line('Year', ['Gasoline', 'Fuel oils', 'Coal n.e.c.', 'Motorized vehicles', 'Transport equipment',
    #                     'Machinery', 'Newsprint paper', 'Printed products'], figsize = (20, 10), title = z)
    #ax.set_xlabel('Year')
    #ax.set_ylabel('Millions of Dollars of Growth')
    #ax.grid(True)
    #ax.legend(['Gasoline', 'Fuel oils', 'Coal n.e.c', 'Motorized vehicles', 'Transport equipment', 'Machinery',
    #            'Newsprint/paper', 'Printed products'])
    
    bx = df.plot.bar('Year', ['Gasoline', 'Fuel oils', 'Coal n.e.c.', 'Motorized vehicles', 'Transport equipment',
                          'Machinery', 'Newsprint paper', 'Printed products'], figsize = (15, 10), title = z)
    bx.set_xlabel('Year')
    bx.set_ylabel('Millions of Dollars')
    bx.grid(True)
    bx.legend(['Gasoline', 'Fuel oils', 'Coal n.e.c', 'Motorized vehicles', 'Transport equipment', 'Machinery',
                'Newsprint/paper', 'Printed products'])
    
    #plt.stackplot(time, gasoline, fuel_oils, coal_nec, motorized_vehicles, transport_equipment, machinery,
    #            newsprint_paper, printed_products)
    

### Methods to Separate Ozone & PM2.5

In [3]:
def createOzoneDataframe(x):
    df = x
    df.drop(['PM2.5_AQI_Value'], axis = 1)
    df.dropna()
    df['ozone_rolling_mean_365'] = df['Ozone_AQI_Value'].rolling(window=90).mean()
    data = pd.DataFrame({'X' : df['ozone_rolling_mean_365'],
                         'T' : df['Date']})
    
    return data

def createParticleDataframe(x):
    df = x
    df.drop(['Ozone_AQI_Value'], axis = 1)
    df.dropna()
    x['particle_rolling_mean_365'] = x['PM2.5_AQI_Value'].rolling(window=90).mean()
    data = pd.DataFrame({'Y' : x['particle_rolling_mean_365'],
                         'T' : x['Date']})
    return data

def merge_plots(x, y):
    data = pd.merge(x, y, how = 'inner', on = 'T')
    return data

def ozone_plot(x):
    ax = x.plot.line('T', 'X')
    
def particle_plot(x):
    ax = x.plot.line('T', 'Y')


## Processing The Data

Here we access the file locations for each metropolitan statistical area and use the methods to create dataframes we can work with so we can plot them.

In [4]:
ny_nj_pa_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/NewYork-Newark-JerseyCity/')
ny_nj_pa = (createDataframe(ny_nj_pa_files))
ny_nj_pa_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/NY-NJ-PA-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

il_in_wi_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Chicago-Naperville-Elgin/')
il_in_wi = (createDataframe(il_in_wi_files))
il_in_wi_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/IL-IN-WI-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

tx_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Dallas-FortWorth-Arlington/')
tx = (createDataframe(tx_files))
tx_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/TX-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

ga_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Atlanta-SandySprings-Roswell/')
ga = (createDataframe(ga_files))
ga_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/GA-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

ma_nh_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Boston-Cambridge-Newton/')
ma_nh = (createDataframe(ma_nh_files))
ma_nh_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/MA-NH-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

mi_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Detroit-Warren-Dearborn/')
mi = (createDataframe(mi_files))
mi_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/MI-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

ca_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/LosAngeles-LongBeach-Anaheim/')
ca = (createDataframe(ca_files))
ca_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/CA-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

fl_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Miami-FortLauderdale-WestPalmBeach/')
fl = (createDataframe(fl_files))
fl_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/FL-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

pa_nj_de_md_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Philadelphia-Camden-Wilmington/')
pa_nj_de_md = (createDataframe(pa_nj_de_md_files))
pa_nj_de_md_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/PA-NJ-DE-MD-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

az_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Phoenix-Mesa-Scottsdale/')
az = (createDataframe(az_files))
az_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/AZ-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

wa_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Seattle-Tacoma-Bellevue/')
wa = (createDataframe(wa_files))
wa_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/WA-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

dc_va_md_wv_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Washington-Arlington-Alexandria/')
dc_va_md_wv = (createDataframe(dc_va_md_wv_files))
dc_va_md_wv_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/DC-VA-MD-WV-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

mn_wi_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Minneapolis-St.Paul-Bloomington/')
mn_wi = (createDataframe(mn_wi_files))
mn_wi_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/MN-WI-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

co_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/Denver-Aurora-Lakewood/')
co = (createDataframe(co_files))
co_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/CO-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

mo_il_files = concatinate('/Users/smaslam/Desktop/airpollution/airpollution/Areas/St.Louis/')
mo_il = (createDataframe(mo_il_files))
mo_il_production = pd.DataFrame(pd.read_csv('/Users/smaslam/Desktop/airpollution/airpollution/Production/MO-IL-Production.csv',
                                         sep = r'\s*,\s*', engine = 'python'))

areas = [ny_nj_pa, il_in_wi, tx, ga, ma_nh, mi, ca, fl, pa_nj_de_md, az, wa, dc_va_md_wv, mn_wi, co, mo_il]

### Testing Different Rolling Mean Through Separate Ozone and Particle Functions

In [5]:
#test = createOzoneDataframe(ny_nj_pa_files)
#test2 = createParticleDataframe(ny_nj_pa_files)
#ozone_plot(test)
#particle_plot(test2)

In [6]:
#time_histogram(ga_files)